# Train a custom YOLO model

### Install Packages

In [ ]:
#TODO Remove this file at some point or move somewhere else

In [ ]:
# %pip install roboflow
# %pip install utralytics
# %pip install python-dotenv

### Import Packages

In [10]:
from roboflow import Roboflow
from ultralytics import YOLO
from dotenv import load_dotenv
import os

In [11]:
# Load enviroment variables
load_dotenv()
ROBOFLOW_API = os.getenv("ROBOFLOW_API")

## 1. Import training dataset from roboflow

In [10]:
#Place download code from roboflow
rf = Roboflow(api_key=ROBOFLOW_API)
project = rf.workspace("test-vlslb").project("car-owtrd-2we06")
version = project.version(1)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

## 2. Train the model

In [13]:
#define the model based on existing Yolo model (in this case the segmentation model)
model_path = r"C:\Users\tomer\OneDrive\Home\Projects\satellite_imagery_interpreter\models\yolo12x.pt" #TODO change to relative path
model = YOLO(model_path)

#define variables
ROBOFLOW_DATA=r"C:\Users\tomer\OneDrive\Home\Projects\satellite_imagery_interpreter\Data\model_training_data\training_data_00\data.yaml" #TODO change to relative path
EPOCH_SIZE=100
IMAGE_SIZE=640

#train the model based on the labeled data from Roboflow
results = model.train(data=ROBOFLOW_DATA ,  epochs=EPOCH_SIZE, imgsz=IMAGE_SIZE)

Ultralytics 8.3.199  Python-3.13.0 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## 3. Validate the model

In [ ]:
# Show performance metrics
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category
metrics.seg.map  # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps  # a list contains map50-95(M) of each category